## Park 1: This notebook will be mainly used for the capstone project for the IBM DATA SCIENCE PROFESSIONAL CERTIFICATE on Coursera

In [170]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [113]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 2: I will be scraping the Wikipedia Page "List of postal codes of Canada: M"
to create a pandas dataframe of Canada's Postal code, Borough and Neighborhoods.

In [114]:
from bs4 import BeautifulSoup

In [115]:
import requests

Scrape the text of the wikipedia page that includes the table we want

In [116]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [117]:
soup = BeautifulSoup(source, 'lxml')

In [118]:
#print(soup.prettify())

Extract the table out of the text

In [119]:
table = soup.find_all('table')[0]
#print(table.prettify())

Convert into a dataframe

In [120]:
df = pd.read_html(str(table))

In [121]:
df = df[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Clean the Datafram

In [122]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [123]:
#Delete Row with Borough = Not Assigned
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [126]:
# Combine Rows with same Postcode/Borough
df2 = df.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(', '.join)
df2['Neighbourhood']=set(df2['Neighbourhood'])


Just in case...

In [127]:
#Replacing Not assigned neighborhood with Borough name
df2["Neighbourhood"].replace('Not assigned', df.Borough, inplace=True)

In [128]:
df2.reset_index()
df2.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Guildwood, Morningside, West Hill"
1,M4A,North York,"Rouge, Malvern"
2,M5A,Downtown Toronto,St. James Town
3,M6A,North York,"Parkdale, Roncesvalles"
4,M7A,Queen's Park,Leaside
5,M9A,Etobicoke,Humewood-Cedarvale
6,M1B,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M3B,North York,"Humber Bay Shores, Mimico South, New Toronto"
8,M4B,East York,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
9,M5B,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


Look at the shape of the dataframe

In [129]:
df2.shape

(103, 3)

We will get the latitude and longitude coordinates of the neighborhoods

In [145]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [146]:
print(coors.shape)
coors.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will now merge the two dataframes

In [149]:
df_temp = df2.set_index('Postcode')
coors_temp = coors.set_index('Postal Code')
df_coors = pd.concat([df_temp, coors_temp], axis=1, join='inner')

In [150]:
df_coors.index.name = 'PostalCode'
df_coors.reset_index(inplace=True)

In [151]:
print(df_coors.shape)
df_coors.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Guildwood, Morningside, West Hill",43.753259,-79.329656
1,M4A,North York,"Rouge, Malvern",43.725882,-79.315572
2,M5A,Downtown Toronto,St. James Town,43.654260,-79.360636
3,M6A,North York,"Parkdale, Roncesvalles",43.718518,-79.464763
4,M7A,Queen's Park,Leaside,43.662301,-79.389494


Explore the Cluster of Neighborhoods in Toronto

In [152]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [154]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [157]:
import folium

Create a map of Toronto with neighborhoods superimposed on top

In [160]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_coors['Latitude'], df_coors['Longitude'], df_coors['PostalCode'], df_coors['Borough'], df_coors['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

 Define Foursquare Credentials and Version

## Insert Foursquare Credentials Here

In [165]:
#Get 1 name of a neighborhood
df_coors.loc[4, 'Neighbourhood']

'Leaside'

In [ ]:
neighborhood_latitude = df_coors.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_coors.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_coors.loc[4, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Now, let's get the top 100 venues that are in Leaside within a radius of 500 meters.

In [168]:
#First, let's create the GET request URL
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XSKY2H0EQZ1H4CV4W0B5HJG2DXGFFKKKCIVSPKW1SHRVTROD&client_secret=3EDCYVT1AJTUTVPRIKNM1EIPCEHDHODA1XFVDGHCJLWPKSW3&ll=43.653963,-79.387207&v=20180605&radius=500&limit=100'

In [169]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5dc126018cbb002cb378f5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [171]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [172]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Textile Museum of Canada,Art Museum,43.654396,-79.386500
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501


In [173]:
#Count the number of venues that were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

72 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [174]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [175]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called canada_venues

canada_venues = getNearbyVenues(names=df_coors['Neighbourhood'],
                                   latitudes=df_coors['Latitude'],
                                   longitudes=df_coors['Longitude']
                                  )




Guildwood, Morningside, West Hill
Rouge, Malvern
St. James Town
Parkdale, Roncesvalles
Leaside
Humewood-Cedarvale
East Birchmount Park, Ionview, Kennedy Park
Humber Bay Shores, Mimico South, New Toronto
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Design Exchange, Toronto Dominion Centre
Stn A PO Boxes 25 The Esplanade
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Northwood Park, York University
Adelaide, King, Richmond
Lawrence Park
Westmount
Dorset Park, Scarborough Town Centre, Wexford Heights
Commerce Court, Victoria Hotel
Weston
High Park, The Junction South
Woodbine Gardens, Parkview Hill
Downsview West
Cabbagetown, St. James Town
Birch Cliff, Cliffside West
Harbord, University of Toronto
Emery, Humberlea
Flemingdon Park, Don Mills South
Caledonia-Fairbanks
Moore Park, S

Let's check the size of the resulting dataframe

In [176]:
print(canada_venues.shape)
canada_venues.head()

(2244, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Guildwood, Morningside, West Hill",43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,"Guildwood, Morningside, West Hill",43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,"Guildwood, Morningside, West Hill",43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,"Rouge, Malvern",43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,"Rouge, Malvern",43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [177]:
canada_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",9,9,9,9,9,9
Agincourt,20,20,20,20,20,20
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",36,36,36,36,36,36
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",1,1,1,1,1,1
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Downsview North, Wilson Heights",21,21,21,21,21,21
Bayview Village,84,84,84,84,84,84
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Berczy Park,19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues


In [178]:
print('There are {} uniques categories.'.format(len(canada_venues['Venue Category'].unique())))

There are 280 uniques categories.


## Analyze Each Neighborhood

In [179]:
# one hot encoding
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]

canada_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's 

In [180]:
canada_onehot.shape

(2244, 280)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [181]:
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's 

In [183]:
canada_grouped.shape

(101, 280)

#### Let's print each neighborhood along with the top 5 most common venues

In [184]:
num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0    Skating Rink  0.22
1     Curling Ice  0.11
2        Pharmacy  0.11
3  Cosmetics Shop  0.11
4            Park  0.11


----Agincourt----
                    venue  freq
0             Coffee Shop  0.10
1          Clothing Store  0.10
2     Sporting Goods Shop  0.10
3             Yoga Studio  0.05
4  Furniture / Home Store  0.05


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2         Pizza Place  0.06
3               Diner  0.06
4  Italian Restaurant  0.06


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                             venue  freq
0                             Park   1.0
1                      Yoga Studio   0.0
2                      Men's Store   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Alde

           venue  freq
0       Bus Line  0.22
1         Bakery  0.22
2    Bus Station  0.11
3   Intersection  0.11
4  Metro Station  0.11


----Downsview Northwest----
                venue  freq
0         Pizza Place  0.11
1  Italian Restaurant  0.06
2           Pet Store  0.06
3                 Pub  0.06
4       Movie Theater  0.06


----Downsview West----
                             venue  freq
0                      Coffee Shop  0.67
1                Korean Restaurant  0.33
2                    Metro Station  0.00
3              Monument / Landmark  0.00
4  Molecular Gastronomy Restaurant  0.00


----Downsview, North Park, Upwood Park----
                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3                      Men's Store   0.0
4  Molecular Gastronomy Restaurant   0.0


----East Birchmount Park, Ionview, Kennedy Park----
                             venue  freq
0      

                 venue  freq
0  American Restaurant  0.33
1        Movie Theater  0.33
2                Motel  0.33
3               Market  0.00
4    Martial Arts Dojo  0.00


----Rouge, Malvern----
                        venue  freq
0       Portuguese Restaurant  0.25
1  Financial or Legal Service  0.25
2                Hockey Arena  0.25
3                 Coffee Shop  0.25
4                 Yoga Studio  0.00


----Runnymede, Swansea----
                             venue  freq
0                       Food Truck  0.33
1                     Home Service  0.33
2                   Baseball Field  0.33
3                      Yoga Studio  0.00
4  Molecular Gastronomy Restaurant  0.00


----Ryerson, Garden District----
         venue  freq
0     Pharmacy  0.12
1       Bakery  0.12
2  Supermarket  0.12
3          Bar  0.06
4  Music Venue  0.06


----Scarborough----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3          

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [185]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [186]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Skating Rink,Pharmacy,Beer Store,Bus Stop,Curling Ice,Cosmetics Shop,Park,Video Store,Construction & Landscaping,Dive Bar
1,Agincourt,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Restaurant,Metro Station,Mexican Restaurant,Dessert Shop,Diner,Chinese Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Café,Coffee Shop,Italian Restaurant,Pizza Place,Diner,Sushi Restaurant,Indie Movie Theater,Metro Station,Food,Ice Cream Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
4,"Alderwood, Long Branch",Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar


##  Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [187]:
# set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [214]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = df_coors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood',how = 'right')

canada_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,"Guildwood, Morningside, West Hill",43.753259,-79.329656,0,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M4A,North York,"Rouge, Malvern",43.725882,-79.315572,2,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,St. James Town,43.654260,-79.360636,2,Coffee Shop,Bakery,Café,Pub,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Cosmetics Shop
3,M6A,North York,"Parkdale, Roncesvalles",43.718518,-79.464763,2,Clothing Store,Women's Store,Shoe Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Event Space,Vietnamese Restaurant,Boutique
4,M7A,Queen's Park,Leaside,43.662301,-79.389494,2,Coffee Shop,Park,Gym,Diner,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café


Finally, let's visualize the resulting clusters

##  Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [215]:
#Cluster 0
canada_merged.loc[canada_merged['Cluster Labels'] == 0, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Fast Food Restaurant,Food & Drink Shop,Park,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
21,York,0,Park,Women's Store,Market,Fast Food Restaurant,Concert Hall,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Comic Shop
35,East York,0,Park,Pizza Place,Convenience Store,Coffee Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
40,North York,0,Park,Airport,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
52,North York,0,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
61,Central Toronto,0,Park,Swim School,Bus Line,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
64,York,0,Park,Convenience Store,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
77,Etobicoke,0,Park,Bus Line,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
83,Central Toronto,0,Gym,Playground,Restaurant,Park,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
85,Scarborough,0,Park,Playground,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [216]:
#Cluster 2
canada_merged.loc[canada_merged['Cluster Labels'] == 1, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,1,Bank,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
46,North York,1,Grocery Store,Park,Shopping Mall,Bank,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
66,North York,1,Park,Convenience Store,Bank,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [217]:
#Cluster 2
canada_merged.loc[canada_merged['Cluster Labels'] == 2, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Financial or Legal Service,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
2,Downtown Toronto,2,Coffee Shop,Bakery,Café,Pub,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Cosmetics Shop
3,North York,2,Clothing Store,Women's Store,Shoe Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Event Space,Vietnamese Restaurant,Boutique
4,Queen's Park,2,Coffee Shop,Park,Gym,Diner,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café
6,Scarborough,2,Fast Food Restaurant,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
7,North York,2,Gym / Fitness Center,Basketball Court,Caribbean Restaurant,Japanese Restaurant,Café,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Event Space,Dive Bar
8,East York,2,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Bank,Pet Store,Pharmacy,Intersection,Athletics & Sports,Gastropub,Donut Shop
9,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Fast Food Restaurant,Restaurant,Japanese Restaurant,Bookstore
10,North York,2,Bakery,Pub,Italian Restaurant,Japanese Restaurant,Women's Store,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
13,North York,2,Gym,Beer Store,Coffee Shop,Asian Restaurant,Fast Food Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop,Japanese Restaurant,Italian Restaurant


In [218]:
#Cluster 3

canada_merged.loc[canada_merged['Cluster Labels'] == 3, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,3,Bar,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


In [221]:
#Cluster 4

canada_merged.loc[canada_merged['Cluster Labels'] == 4, canada_merged.columns[[1] + list(range(5, canada_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Cafeteria,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


## Visualize the cluster on a Map

In [220]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Neighbourhood'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

With a quick analysis, we can see that majority of the neighborhoods, especially the ones along the coast are part of cluster 2, characterized by a high number of coffee shops and restaurants, as opposed to parks and playgrounds.